L3_demo_05_database_toolkit

In [ ]:
from typing import List
import sqlalchemy
from sqlalchemy.engine.base import Engine
from sqlalchemy import text, create_engine
import pandas as pd
from langchain_core.tools import tool
from langchain_core.runnables.config import RunnableConfig

In [ ]:
db_engine = create_engine(f"sqlite:///sales.db")

In [ ]:
inspector = sqlalchemy.inspect(db_engine)

In [ ]:
inspector.get_table_names()

In [ ]:
table_name = "sales"
inspector.get_columns(table_name)

In [ ]:
schema = inspector.get_columns(table_name)
column_names = [column["name"] for column in schema]
column_names

In [ ]:
sql = f"SELECT * FROM {table_name} LIMIT 10"

In [ ]:
with db_engine.begin() as connection:
    answer = connection.execute(text(sql)).fetchall()

In [ ]:
answer

In [ ]:
pd.DataFrame(answer, columns=column_names)

Tools

In [ ]:
@tool
def list_tables_tool(config: RunnableConfig) -> List[str]:
    """
    List all tables in database
    """
    db_engine:Engine = config.get("configurable", {}).get("db_engine")
    inspector = sqlalchemy.inspect(db_engine)

    return inspector.get_table_names()

In [ ]:
@tool
def get_table_schema_tool(table_name:str, config: RunnableConfig) -> List[str]:
    """
    Get schema information about a table. Returns a list of dictionaries.
    - name is the column name
    - type is the column type
    - nullable is whether the column is nullable or not
    - default is the default value of the column
    - primary_key is whether the column is a primary key or not

    Args:
        table_name (str): Table name
    """
    db_engine:Engine = config.get("configurable", {}).get("db_engine")
    inspector = sqlalchemy.inspect(db_engine)

    return inspector.get_columns(table_name)

In [ ]:
@tool
def execute_sql_tool(query:str, config: RunnableConfig) -> int:
    """
    Execute SQL query and return result. 
    This will automatically connect to the database and execute the query.
    However, if the query is not valid, an error will be raised

    Args:
        query (str): SQL query
    """
    db_engine:Engine = config.get("configurable", {}).get("db_engine")
    with db_engine.begin() as connection:
        answer = connection.execute(text(query)).fetchall()

    return answer

In [ ]:
db_engine = create_engine(f"sqlite:///sales.db")

In [ ]:
config = {'configurable':{'db_engine': db_engine}}

In [ ]:
tables = list_tables_tool.invoke({}, config)
tables

In [ ]:
schemas = {
    table: get_table_schema_tool.invoke({
            'table_name': table
        }, config) 
    for table in tables
}
schemas

In [ ]:
sql = f"SELECT * FROM {tables[0]} LIMIT 10"
result = execute_sql_tool.invoke({'query': sql}, config)
result

## Exercise - Create a Text2SQL Agent - STARTER
In this exercise, you’ll build an AI agent capable of converting natural language questions into SQL queries, executing them, and returning the results.

Challenge

You’re building a Text2SQL assistant for a Sales Dashboard. The agent should:

Parse user questions.
Identify the relevant tables and columns.
Generate the corresponding SQL query.
Execute the query and return the result.
This enables non-technical users to ask questions in plain English and get data insights without writing SQL.

## 0. Import the necessary libs


In [1]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from sqlalchemy import create_engine

# this next stuff was defined in a previous demo: see above
from sql_toolkit import (
    list_tables_tool, 
    get_table_schema_tool, 
    execute_sql_tool
)

ModuleNotFoundError: No module named 'sql_toolkit'

1. Instantiate Chat Model with your API Key
To be able to connect with OpenAI, you need to instantiate an ChatOpenAI client passing your OpenAI key.

You can pass the api_key argument directly.

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
    api_key="voc-",
)

In [ ]:
from dotenv import load_dotenv
load_dotenv()
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.0,
)

2. Initialize the State Graph
You can either chose from creating it with TypedDict, Pydantic or using the MessagesState. It should store:

user_query: The user's natural language question.
messages: The conversation history.

In [ ]:
class State(MessagesState):
    user_query: str

workflow = StateGraph(State)

3. Define the tools available
We created the following tools:

list_tables_tool: Lists tables in the database.,
get_table_schema_tool: Retrieves the schema of a specific table,
execute_sql_tool: Executes the generated SQL query.
They are all accessible in the sql_toolkit.py. It's already imported. You need to add them to the tools list, then add the tools list to a node and bind the tools to the llm.

In [ ]:
dba_tools = [list_tables_tool, get_table_schema_tool, execute_sql_tool]

In [ ]:
workflow.add_node("dba_tools", ToolNode(dba_tools))

In [ ]:
dba_llm = llm.bind_tools(dba_tools, tool_choice="auto")

4. The agent node
Before creating the dba_agent node, you need to build the initial messages to start the conversation. So, we'll first create the messages builder node.

The llm you'll use has already the tools bound to it, so the message will have a content or a tool_calls.

In [ ]:
def messages_builder(state: State):
    dba_sys_msg = (
        "You are a Sr. SQL developer tasked with generating SQL queries. Perform the following steps:\n"
        "First, find out the appropriate table name based on all tables. "
        "Then get the table's schema to understand the columns. "
        "With the table name and the schema, generate the ANSI SQL query you think is applicable to the user question. "
        "Finally, use a tool to execute the above SQL query and output the result based on the user question."
    )
    messages = [
        SystemMessage(dba_sys_msg),
        HumanMessage(state["user_query"])
    ]
    return {"messages": messages}

In [ ]:
def dba_agent(state: State):
   ai_message = dba_llm.invoke(state["messages"])
   ai_message.name = "dba_agent"
   return {"messages": ai_message}

In [ ]:
workflow.add_node("messages_builder", messages_builder)
workflow.add_node("dba_agent", dba_agent)

In [ ]:
5. Edges
To enable the cycle, you need to have a routing function. This function:

Inspects the conversation history.
Decides whether to call a tool or end the workflow.
Then, create the nodes:

After the START entrypoint, you need to move to messages_builder
From messages_builder, to the dba_agent
The dba_agent should call the routing_function within a conditional edge
If the response is a tool_call, send to dba_tools node
Else, terminates in the END termination node.


In [ ]:
def should_continue(state: State):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "dba_tools"
    return END

In [ ]:
workflow.add_edge(START, "messages_builder")
workflow.add_edge("messages_builder", "dba_agent")
workflow.add_conditional_edges(
    source="dba_agent", 
    path=should_continue, 
    path_map=["dba_tools", END]
)
workflow.add_edge("dba_tools", "dba_agent")

6. Compile and Run
Compile your graph, display it and run for inputs and config passed in the invoke() method.

In [ ]:
react_graph = workflow.compile()

In [ ]:
display(
    Image(
        react_graph.get_graph().draw_mermaid_png()
    )
)

In [ ]:
db_engine = create_engine(f"sqlite:///sales.db")

In [ ]:
config = {
    "configurable": {
        "db_engine": db_engine
    }
}

In [ ]:
inputs = {
    "user_query": "How many Dell XPS 15 were sold?"
}

In [ ]:
messages = react_graph.invoke(
    input=inputs, 
    config=config,
)

In [ ]:
for m in messages['messages']:
    m.pretty_print()

7. Experiment
Now that you understood how it works, experiment with new things.

Use the tools to understand what you have in the sales.db
Ask other questions to it
Use your own database